# Alert me when news happens

In this section, we're going to talk about how you can use Python to alert you when news happens. Remember when the LA Times made a bot to write [breaking news stories about earthquakes](http://www.latimes.com/local/lanow/la-me-ln-earthquake-magnitude-30-near-cobb-calif-20161217-story.html)? Yeah me too. That was rad.

The Omaha World-Herald has a [_complicated_ data pipeline](https://github.com/OWH-projects/nadc_data) for processing campaign finance reports. If you had developed this and wanted to get an alert any time there was a donation over $100,000, or from the governor, or whatever, you could do that.

Python has a [built-in module for sending emails](https://docs.python.org/3/library/email-examples.html). Many apps and web services (like Twitter and Slack) have an [Application Programming Interface](http://www.webopedia.com/TERM/A/API.html) (API) that your script can shake hands with.

Our goal today is to send a message to Slack.

### Scenario 1
Every Monday, you download an updated spreadsheet of bids from the city you cover. Every Monday, you open that spreadsheet, filter down to the one category you're interested in, sort descending and make a note of any bids over $100,000.

**What if instead** you just got a Slack message that compiled all the six-figure bids for you?

_Python can make that happen._

### Scenario 2

You cover a nuclear plant. You've written a script to check the NRC website every day to see if your plant has had any "[reportable events](https://www.nrc.gov/reading-rm/doc-collections/event-status/event/)." You are tired of clicking through the website every day to search for reports about your plant. You'd rather just get an email.

_Python!_

## Let's put a thing on Slack

We're going to post a simple message to an IRE Slack channel. I've already set up an [incoming webhook URL](https://api.slack.com/incoming-webhooks) to receive the message.

Now all we need to do is send ("POST") a bit of data to this URL.

Our script will use three modules:

### The `os` module

For security, the Slack webhook URL that I set up is (hopefully!) available on your computer as an "environmental variable." You don't need to know what that means -- just, it would be bad if it were public, and the code you are reading now is stored in a [public GitHub repository](https://github.com/ireapps/djnf-coding).

Python's [`os` module](https://docs.python.org/3/library/os.html) gives your script an interface to your operating system. In our case, we just need to grab the URL string from where it's stored on your computer.

Our variable is called `IRE_SLACK_WEBHOOK`. You'd use `os.environ` to get a handle to it:

```python
import os

slack_webhook = os.environ['IRE_SLACK_WEBHOOK']
```


### The `json` module

JSON stands for JavaScript Object Notation. It's a common format for exchanging data on the web, especially when you're interacting with an API.

Python has a module called [`json`](https://docs.python.org/3/library/json.html) for encoding and decoding JSON data.

To turn a Python dictionary into a JSON object, you could do something like this:

```python
import json

my_dict = {'a': 1, 'b': 2, 'c': 3}

# https://docs.python.org/3/library/json.html#json.dumps
# dumps() turns it into a string of valid JSON
to_json = json.dumps(my_dict)
```

### The `requests` module

[`requests`](http://docs.python-requests.org/en/master/) is a suuuuuper handy third-party module for sending and receiving data over HTTP -- much easier to use than Python's built-in tools for HTTP traffic.

It has a `post` method for sending data -- just hand it the URL you want to send it to and set your string of data as the value of the `data` key, like this:

```python
import requests
import json

payload = {'a': 1, 'b': 2, 'c': 3}

r = requests.post('http://example.com/api/endpoint', data=json.dumps(payload))
```

---

_OK let's do this thing._

The URL we're sending data to is expecting a JSON object with key/value pairs that look something like this:

```json
{
  "channel": "djnf-alerts",
  "username": "DJNF Bot",
  "icon_emoji": ":rocket:"
  "text": "This is a message!"
}
```

Kind of looks like a Python dictionary, no? (Yes.) We're going to make a Python dictionary, call `json.dumps()` on it to convert it to a string of JSON, then send it to that URL.

Let's pretend that we have a Python script to summarize opening-day MLB stats for us (lol). Yeah, we're gonna reuse this sucker:

```python
import csv
import statistics

with open('../data/mlb.csv', 'r') as infile:
    reader = csv.DictReader(infile)
    salaries = [float(row['SALARY']) for row in reader]

    count = len(salaries)
    total = sum(salaries)
    average = statistics.mean(salaries)
    median = statistics.median(salaries)

    summary = 'MLB opening day roster 2016 stats:\n- Players: {}\n' \
              '- Total payroll: {}\n- Average: {}\n' \
              '- Median: {}'.format(count, total, average, median)

    print(summary)
```

Instead of printing the summary, you could send it to slack -- the `summary` variable can be the value for the `text` key in the JSON string that we send to Slack.

**Send a message to the `djnf-alerts` Slack channel using a username and [emoji of your choosing](https://www.webpagefx.com/tools/emoji-cheat-sheet/) (don't forget the colons around the emoji!). You can either do the MLB stats or send us a custom message, whatever feels right in the moment.**